In [1]:
# libraries from finetuning_parameters.py
from finetuning_parameters import get_args
from future.baseline_trainer import BaselineTuner
from future.modules import ptl2classes
from future.hooks import EvaluationRecorder

from data_loader.wrap_sampler import wrap_sampler
import data_loader.task_configs as task_configs
import data_loader.data_configs as data_configs
from future.collocate_fns import task2collocate_fn

import utils.checkpoint as checkpoint
import utils.logging as logging

import torch
import random
import os

# libraries from future/base.py
from torch.utils.data import SequentialSampler, RandomSampler
from future.hooks import EvaluationRecorder
import utils.eval_meters as eval_meters
from seqeval.metrics import f1_score as f1_score_tagging
import torch

# libraries from future/baseline_trainer.py
import torch
import torch.nn as nn
import numpy as np
from copy import deepcopy
from future.base import BaseTrainer
from future.hooks.base_hook import HookContainer
from future.hooks import EvaluationRecorder
from torch.utils.data import RandomSampler
from collections import defaultdict, Counter
from tqdm import tqdm

# and so on..
from finetuning_baseline import init_config, init_task, init_hooks

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

`fused_weight_gradient_mlp_cuda` module not found. gradient accumulation fusion with weight gradient computation disabled.


In [2]:
parser = get_args()
conf = parser.parse_args(args=[])
print(conf)

Namespace(checkpoint='./checkpoint_baseline', data_path='./data/download', dataset_name='mldoc', eval_every_batch=10, eval_languages='english', experiment='debug', finetune_batch_size=32, finetune_epochs=5, finetune_lr=5e-05, inference_batch_size=512, local_rank=-1, manual_seed=3, max_seq_len=128, model='bert-base-multilingual-cased', override=True, ptl='bert', summary_freq=100, time_stamp=None, track_time=True, train_fast=True, trn_languages='japanese', world='0')


In [3]:
conf.dataset_name = 'panx'
conf.trn_languages = 'english'
if conf.dataset_name == 'udpos':
    
    x=os.popen('ls ./buckets/udpos/1-shots').read()
    a=''
    for i in range(len(x)//3):
        lang = x[3*i] + x[3*i+1]
        if lang in data_configs.abbre2language:
            if data_configs.abbre2language[lang] in ['french', 'german', 'indonesian', 'italian', 'russian']:
                continue
            a += data_configs.abbre2language[lang] +','
    conf.eval_languages = a[:-1]
    path='udpos/debug/1646813474_model_task-udpos_flr-1.0E-05_ftbs-32_ftepcs-10_sd-3_trnfast-False_evalevery-500_tlang-en_vlang-en'    

elif conf.dataset_name == 'panx':

    x=os.popen('ls ./buckets/panx/1-shots').read()
    a=''
    for i in range(len(x)//3):
        lang= x[3*i] + x[3*i+1]
        if lang in data_configs.abbre2language:
            if data_configs.abbre2language[lang] in ['arabic', 'burmese', 'chinese', 'georgian', 'greek', 'hebrew', 'japanese','korean','malay','malayalam', 'tamil','thai','urdu']:
                continue
            a += data_configs.abbre2language[lang] +','
    conf.eval_languages = a[:-1]
    path='panx/debug/1646876250_model_task-panx_flr-1.0E-05_ftbs-32_ftepcs-10_sd-3_trnfast-False_evalevery-500_tlang-en_vlang-en'

conf.checkpoint_root = os.path.join(conf.checkpoint, path)
conf.finetune_epochs = 10
conf.finetune_batch_size = 256
conf.eval_every_batch = 500
conf.override = False
conf.train_fast = False
conf.world = '0'
conf.finetune_lr = 1e-5
conf.vocab_size= 119547 ## stopgap
print(conf)


Namespace(checkpoint='./checkpoint_baseline', checkpoint_root='./checkpoint_baseline/panx/debug/1646876250_model_task-panx_flr-1.0E-05_ftbs-32_ftepcs-10_sd-3_trnfast-False_evalevery-500_tlang-en_vlang-en', data_path='./data/download', dataset_name='panx', eval_every_batch=500, eval_languages='afrikaans,bulgarian,bengali,german,english,spanish,estonian,basque,persian,finnish,french,hindi,hungarian,indonesian,italian,javanese,kazakh,marathi,dutch,portuguese,russian,swahili,telugu,tagalog,turkish,vietnamese,yoruba', experiment='debug', finetune_batch_size=256, finetune_epochs=10, finetune_lr=1e-05, inference_batch_size=512, local_rank=-1, manual_seed=3, max_seq_len=128, model='bert-base-multilingual-cased', override=False, ptl='bert', summary_freq=100, time_stamp=None, track_time=True, train_fast=False, trn_languages='english', vocab_size=119547, world='0')


In [4]:
init_config(conf)
model, tokenizer, data_iter, metric_name, collocate_batch_fn = init_task(conf)

data_iter

./checkpoint_baseline/panx/debug/1646879050_model_task-panx_flr-1.0E-05_ftbs-256_ftepcs-10_sd-3_trnfast-False_evalevery-500_tlang-en_vlang-27


parameters: 
	override	False
	experiment	debug
	ptl	bert
	model	bert-base-multilingual-cased
	dataset_name	panx
	max_seq_len	128
	trn_languages	['english']
	eval_languages	['afrikaans', 'bulgarian', 'bengali', 'german', 'english', 'spanish', 'estonian', 'basque', 'persian', 'finnish', 'french', 'hindi', 'hungarian', 'indonesian', 'italian', 'javanese', 'kazakh', 'marathi', 'dutch', 'portuguese', 'russian', 'swahili', 'telugu', 'tagalog', 'turkish', 'vietnamese', 'yoruba']
	finetune_epochs	10
	eval_every_batch	500
	finetune_batch_size	256
	finetune_lr	1e-05
	inference_batch_size	512
	data_path	./data/download
	checkpoint	./checkpoint_baseline
	manual_seed	3
	summary_freq	100
	time_stamp	None
	train_fast	False
	track_time	True
	world	[0]
	local_rank	-1
	checkpoint_root	./checkpoint_baseline/panx/debug/1646879050_model_task-panx_flr-1.0E-05_ftbs-2

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceTagging: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceTagging from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceTagging from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceTagging were not initialized from the model checkpoint at bert-base-multil

[INFO] loading cached dataset for panx-afrikaans_trn.
[INFO] loading cached dataset for panx-afrikaans_val.
[INFO] loading cached dataset for panx-afrikaans_tst.
[INFO] loading cached dataset for panx-basque_trn.
[INFO] loading cached dataset for panx-basque_val.
[INFO] loading cached dataset for panx-basque_tst.
[INFO] loading cached dataset for panx-bengali_trn.
[INFO] loading cached dataset for panx-bengali_val.
[INFO] loading cached dataset for panx-bengali_tst.
[INFO] loading cached dataset for panx-bulgarian_trn.
[INFO] loading cached dataset for panx-bulgarian_val.
[INFO] loading cached dataset for panx-bulgarian_tst.
[INFO] loading cached dataset for panx-dutch_trn.
[INFO] loading cached dataset for panx-dutch_val.
[INFO] loading cached dataset for panx-dutch_tst.
[INFO] loading cached dataset for panx-english_trn.
[INFO] loading cached dataset for panx-english_val.
[INFO] loading cached dataset for panx-english_tst.
[INFO] loading cached dataset for panx-estonian_trn.
[INFO] l

{'afrikaans': <data_loader.data_iters.TaggingDataIter at 0x7fa5099c3b00>,
 'basque': <data_loader.data_iters.TaggingDataIter at 0x7fa507efe128>,
 'bengali': <data_loader.data_iters.TaggingDataIter at 0x7fa51b4b69e8>,
 'bulgarian': <data_loader.data_iters.TaggingDataIter at 0x7fa50108d828>,
 'dutch': <data_loader.data_iters.TaggingDataIter at 0x7fa519c80240>,
 'english': <data_loader.data_iters.TaggingDataIter at 0x7fa51ae76978>,
 'estonian': <data_loader.data_iters.TaggingDataIter at 0x7fa517ac42e8>,
 'finnish': <data_loader.data_iters.TaggingDataIter at 0x7fa51b1a2358>,
 'french': <data_loader.data_iters.TaggingDataIter at 0x7fa5007492e8>,
 'german': <data_loader.data_iters.TaggingDataIter at 0x7fa4fff65dd8>,
 'hindi': <data_loader.data_iters.TaggingDataIter at 0x7fa519f0e278>,
 'hungarian': <data_loader.data_iters.TaggingDataIter at 0x7fa4fe51b7b8>,
 'indonesian': <data_loader.data_iters.TaggingDataIter at 0x7fa7e7c0c0b8>,
 'italian': <data_loader.data_iters.TaggingDataIter at 0x7fa7

In [5]:
adapt_loaders = {}
for language, language_dataset in data_iter.items():
    # NOTE: the sample dataset are refered
    adapt_loaders[language] = wrap_sampler(
        trn_batch_size=conf.finetune_batch_size,
        infer_batch_size=conf.inference_batch_size,
        language=language,
        language_dataset=language_dataset,
    )
hooks = init_hooks(conf, metric_name)

In [6]:
adapt_loaders

{'afrikaans': <data_loader.data_iters.TaggingDataIter at 0x7fa5099c3b00>,
 'basque': <data_loader.data_iters.TaggingDataIter at 0x7fa507efe128>,
 'bengali': <data_loader.data_iters.TaggingDataIter at 0x7fa51b4b69e8>,
 'bulgarian': <data_loader.data_iters.TaggingDataIter at 0x7fa50108d828>,
 'dutch': <data_loader.data_iters.TaggingDataIter at 0x7fa519c80240>,
 'english': <data_loader.data_iters.TaggingDataIter at 0x7fa51ae76978>,
 'estonian': <data_loader.data_iters.TaggingDataIter at 0x7fa517ac42e8>,
 'finnish': <data_loader.data_iters.TaggingDataIter at 0x7fa51b1a2358>,
 'french': <data_loader.data_iters.TaggingDataIter at 0x7fa5007492e8>,
 'german': <data_loader.data_iters.TaggingDataIter at 0x7fa4fff65dd8>,
 'hindi': <data_loader.data_iters.TaggingDataIter at 0x7fa519f0e278>,
 'hungarian': <data_loader.data_iters.TaggingDataIter at 0x7fa4fe51b7b8>,
 'indonesian': <data_loader.data_iters.TaggingDataIter at 0x7fa7e7c0c0b8>,
 'italian': <data_loader.data_iters.TaggingDataIter at 0x7fa7

In [7]:
trainer = BaselineTuner(
        conf, collocate_batch_fn=collocate_batch_fn, logger=conf.logger
    )
trainer.conf.eval_languages

2022-03-10 02:25:03	Init trainer.


['afrikaans',
 'bulgarian',
 'bengali',
 'german',
 'english',
 'spanish',
 'estonian',
 'basque',
 'persian',
 'finnish',
 'french',
 'hindi',
 'hungarian',
 'indonesian',
 'italian',
 'javanese',
 'kazakh',
 'marathi',
 'dutch',
 'portuguese',
 'russian',
 'swahili',
 'telugu',
 'tagalog',
 'turkish',
 'vietnamese',
 'yoruba']

In [8]:
import pickle

def save_pickle(file, data):
    with open(file, 'wb') as f:
        pickle.dump(data, f)
        
def load_pickle(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [9]:
if conf.dataset_name == 'udpos':  # pos-tagging
    PATH='./checkpoint_baseline/udpos/debug/1646813474_model_task-udpos_flr-1.0E-05_ftbs-32_ftepcs-10_sd-3_trnfast-False_evalevery-500_tlang-en_vlang-en/state_dicts/best_state.pt'
elif conf.dataset_name == 'panx':  # ner
    PATH='./checkpoint_baseline/panx/debug/1646876250_model_task-panx_flr-1.0E-05_ftbs-32_ftepcs-10_sd-3_trnfast-False_evalevery-500_tlang-en_vlang-en/state_dicts/best_state.pt'
model.load_state_dict(torch.load(PATH)['best_state_dict'], strict=True)

<All keys matched successfully>

In [12]:
# trainer.train
opt, model = trainer._init_model_opt(model)
trainer.model = model
trainer.model.eval()

# for epoch_index in tqdm(range(1, 1 + 1)):
    
for language in trainer.conf.eval_languages: # trn_languages
    print ("Start language-{}".format(language))
    labels = np.empty((0,))
    features = np.empty((0, 768))
    trn_iters = []
    egs = adapt_loaders[language].val_egs # trn_egs
    trn_iters.append(iter(egs))

    batches_per_epoch = max(len(ti) for ti in trn_iters)
    for batch_index in range(1, batches_per_epoch + 1):
        trn_loss = []
        for ti in trn_iters:
            try:
                batched = next(ti)
            except StopIteration:
                continue
            batched, golds, uids, _golds_tagging = trainer.collocate_batch_fn(
                batched
            )
            with torch.no_grad():
                hidden = trainer.model.get_last_hidden(**batched)
                if conf.dataset_name in ["udpos", "panx"]:
                    hidden = hidden[batched["if_tgts"]]
                labels = np.concatenate((labels, golds.cpu()))
                features = np.concatenate((features, hidden.cpu()), axis=0)
    
    os.makedirs('./stats/{}'.format(conf.dataset_name), exist_ok=True)
    
    output_dict = {}
    if conf.dataset_name == 'udpos':
        label_len = np.float64(np.unique(range(17)))
    elif conf.dataset_name == 'panx':
        label_len = np.float64(np.unique(range(7)))

    for i in label_len: # stopgap
        feature = features[labels == i]
        mean = np.mean(feature, axis=0)
        cov = np.cov(feature.T)
        output_dict[str(int(i))] = [{"mean": mean, "cov": cov}]
    save_pickle('./stats/{}/{}.pkl'.format(conf.dataset_name, language), output_dict)

Initialize Optimizer and Model
Start language-afrikaans
Start language-bulgarian
Start language-bengali
Start language-german
Start language-english
Start language-spanish
Start language-estonian
Start language-basque
Start language-persian
Start language-finnish
Start language-french
Start language-hindi
Start language-hungarian
Start language-indonesian
Start language-italian
Start language-javanese
Start language-kazakh
Start language-marathi
Start language-dutch
Start language-portuguese
Start language-russian
Start language-swahili
Start language-telugu
Start language-tagalog
Start language-turkish
Start language-vietnamese
Start language-yoruba


# Class - Class

In [13]:
from numpy import dot
from numpy.linalg import norm

def cov_sim(r1, r2):
    result = np.matmul(r1, r2)
    result /= norm(r1) * norm(r2)
    return result

base_infos = load_pickle('./stats/{}/english.pkl'.format(conf.dataset_name))
for i in list(base_infos.keys()):
    print ('Start {}'.format(i))
    base_mean = base_infos[i][0]['mean']
    base_cov = base_infos[i][0]['cov'].flatten()
    for language in trainer.conf.eval_languages:
        infos = load_pickle('./stats/{}/{}.pkl'.format(conf.dataset_name, language))
        mean = infos[i][0]['mean']
        cov = infos[i][0]['cov'].flatten()
        
        cos_sim = dot(base_mean, mean)
        cos_sim /= (norm(base_mean)*norm(mean))

        cos_sim2 = dot(base_cov, cov)
        cos_sim2 /= (norm(base_cov)*norm(cov))
        # cos_sim2 = cov_sim(base_cov, cov)
        print (language, round(cos_sim, 4), round(cos_sim2, 4))
    
    print ('')

Start 0
afrikaans 0.9803 0.8008
bulgarian 0.9841 0.8509
bengali 0.9205 0.7176
german 0.9862 0.8685
english 1.0 1.0
spanish 0.9781 0.7756
estonian 0.9455 0.6421
basque 0.8994 0.5395
persian 0.47 0.6359
finnish 0.8654 0.5607
french 0.9905 0.9138
hindi 0.9697 0.7934
hungarian 0.9328 0.5704
indonesian 0.8098 0.5337
italian 0.989 0.879
javanese 0.969 0.6919
kazakh 0.8623 0.4465
marathi 0.8909 0.5169
dutch 0.9907 0.89
portuguese 0.9932 0.899
russian 0.9575 0.804
swahili 0.9884 0.8151
telugu 0.9109 0.6005
tagalog 0.9794 0.825
turkish 0.9406 0.569
vietnamese 0.9792 0.7825
yoruba 0.8285 0.5034

Start 1
afrikaans 0.9616 0.7136
bulgarian 0.9598 0.8688
bengali 0.9438 0.8512
german 0.9733 0.8521
english 1.0 1.0
spanish 0.9356 0.6195
estonian 0.9702 0.8341
basque 0.9515 0.7107
persian 0.6768 0.5252
finnish 0.9613 0.7059
french 0.988 0.8046
hindi 0.9375 0.836
hungarian 0.9754 0.7418
indonesian 0.9497 0.6509
italian 0.9914 0.8057
javanese 0.9524 0.725
kazakh 0.8677 0.6888
marathi 0.9222 0.796
dutch 0.

In [ ]:
##udpos pos-tagging
# Start 0
# afrikaans 0.9117 0.6977
# arabic 0.687 0.5697
# bulgarian 0.8027 0.5466
# greek 0.8415 0.5678
# english 1.0 1.0
# spanish 0.8469 0.7764
# estonian 0.758 0.6407
# basque 0.6311 0.5661
# persian 0.7985 0.8265
# finnish 0.743 0.564
# hebrew 0.8008 0.6932
# hindi 0.8975 0.8189
# hungarian 0.806 0.688
# japanese 0.85 0.6095
# korean 0.7511 0.4624
# marathi 0.746 0.422
# dutch 0.913 0.5871
# portuguese 0.8815 0.7442
# tamil 0.867 0.4871
# telugu 0.8287 0.2978
# turkish 0.8232 0.71
# urdu 0.8599 0.784
# vietnamese 0.6642 0.7699
# chinese 0.8729 0.7541

# Start 1
# afrikaans 0.9296 0.5618
# arabic 0.9126 0.6452
# bulgarian 0.9146 0.647
# greek 0.9099 0.4879
# english 1.0 1.0
# spanish 0.835 0.561
# estonian 0.218 0.3592
# basque 0.1538 0.2991
# persian 0.8763 0.3401
# finnish 0.2511 0.4332
# hebrew 0.9111 0.6305
# hindi 0.4684 0.2906
# hungarian 0.2232 0.2594
# japanese 0.3855 0.2739
# korean 0.3105 0.3777
# marathi nan nan
# dutch 0.9307 0.6079
# portuguese 0.7944 0.4434
# tamil 0.1912 0.2007
# telugu 0.1385 0.1989
# turkish 0.3008 0.3614
# urdu 0.3078 0.2755
# vietnamese 0.8734 0.6496
# chinese 0.6481 0.5042

# Start 2
# afrikaans 0.8792 0.6146
# arabic 0.6982 0.3588
# bulgarian 0.9064 0.6711
# greek 0.8824 0.7189
# english 1.0 1.0
# spanish 0.9041 0.6277
# estonian 0.8022 0.6267
# basque 0.7417 0.5253
# persian 0.8865 0.6984
# finnish 0.8406 0.6593
# hebrew 0.8321 0.5428
# hindi 0.7935 0.5081
# hungarian 0.8222 0.5602
# japanese 0.8182 0.5231
# korean 0.7321 0.5626
# marathi 0.4717 0.3276
# dutch 0.9065 0.6356
# portuguese 0.9155 0.6262
# tamil 0.8201 0.4325
# telugu 0.8414 0.3405
# turkish 0.813 0.5898
# urdu 0.7048 0.4766
# vietnamese nan nan
# chinese 0.7276 0.4045

# Start 3
# afrikaans 0.8606 0.6269
# arabic 0.7286 0.3919
# bulgarian 0.7794 0.2117
# greek 0.6783 0.2185
# english 1.0 1.0
# spanish 0.8983 0.6871
# estonian 0.8712 0.6224
# basque 0.4068 0.3698
# persian 0.6687 0.4977
# finnish 0.8468 0.4996
# hebrew 0.7918 0.4683
# hindi 0.5152 0.2769
# hungarian 0.7095 0.2126
# japanese 0.0947 0.2722
# korean 0.2877 0.2616
# marathi 0.41 0.2269
# dutch 0.8979 0.6675
# portuguese 0.9051 0.6633
# tamil 0.3229 0.2709
# telugu nan nan
# turkish 0.2047 0.2394
# urdu 0.4398 0.2557
# vietnamese 0.748 0.1828
# chinese 0.7748 0.2802

# Start 4
# afrikaans 0.9499 0.5044
# arabic 0.3599 0.2982
# bulgarian 0.9507 0.5916
# greek 0.9463 0.5773
# english 1.0 1.0
# spanish 0.9272 0.5791
# estonian 0.9223 0.5924
# basque 0.8654 0.4344
# persian 0.9259 0.5846
# finnish 0.9424 0.6084
# hebrew 0.8857 0.4319
# hindi 0.9435 0.5298
# hungarian 0.7798 0.4173
# japanese 0.4941 0.3468
# korean 0.7876 0.4646
# marathi 0.8634 0.3236
# dutch 0.9309 0.6364
# portuguese 0.9038 0.2777
# tamil 0.9136 0.2445
# telugu 0.6092 0.1004
# turkish 0.7984 0.425
# urdu 0.919 0.5206
# vietnamese 0.4709 0.3718
# chinese 0.9385 0.4285

# Start 5
# afrikaans 0.8486 0.3812
# arabic 0.2535 0.4464
# bulgarian 0.642 0.4998
# greek 0.81 0.2708
# english 1.0 1.0
# spanish 0.8432 0.6684
# estonian 0.4567 0.3489
# basque 0.0678 0.3281
# persian 0.8465 0.5028
# finnish 0.411 0.4271
# hebrew 0.6372 0.2839
# hindi 0.6257 0.3222
# hungarian 0.7905 0.3791
# japanese 0.7447 0.4587
# korean 0.7462 0.4933
# marathi 0.6095 0.3807
# dutch 0.8666 0.3699
# portuguese 0.831 0.5874
# tamil 0.7063 0.1204
# telugu 0.8235 0.2559
# turkish 0.7393 0.4257
# urdu 0.5995 0.3525
# vietnamese 0.7671 0.2516
# chinese 0.7335 0.2658

# Start 6
# afrikaans nan nan
# arabic 0.1507 nan
# bulgarian 0.7275 nan
# greek nan nan
# english 1.0 1.0
# spanish 0.4738 0.2631
# estonian 0.7741 0.5106
# basque 0.4505 0.2797
# persian 0.6013 0.4717
# finnish 0.7732 0.5632
# hebrew 0.3259 nan
# hindi nan nan
# hungarian 0.1199 nan
# japanese nan nan
# korean 0.5799 0.1725
# marathi 0.7191 nan
# dutch 0.7342 0.3334
# portuguese 0.5143 0.2051
# tamil nan nan
# telugu 0.6985 0.1959
# turkish 0.7456 0.5891
# urdu nan nan
# vietnamese 0.6166 0.3642
# chinese nan nan

# Start 7
# afrikaans 0.964 0.8514
# arabic 0.8494 0.8187
# bulgarian 0.9331 0.7192
# greek 0.9091 0.7731
# english 1.0 1.0
# spanish 0.9381 0.8322
# estonian 0.8331 0.6158
# basque 0.7853 0.5956
# persian 0.8903 0.8327
# finnish 0.8103 0.5566
# hebrew 0.9171 0.8191
# hindi 0.936 0.879
# hungarian 0.7257 0.3589
# japanese 0.8916 0.8321
# korean 0.683 0.5238
# marathi 0.7978 0.4731
# dutch 0.9577 0.8677
# portuguese 0.9491 0.8653
# tamil 0.8139 0.5253
# telugu 0.82 0.5607
# turkish 0.7953 0.6033
# urdu 0.8947 0.8346
# vietnamese 0.8118 0.7704
# chinese 0.8896 0.8145

# Start 8
# afrikaans 0.5089 0.192
# arabic 0.9799 0.5492
# bulgarian 0.9826 0.5499
# greek 0.9177 0.4227
# english 1.0 1.0
# spanish 0.9885 0.6765
# estonian 0.9781 0.5361
# basque 0.7686 0.431
# persian 0.9252 0.3643
# finnish 0.965 0.5126
# hebrew 0.9372 0.6163
# hindi 0.916 0.4213
# hungarian 0.9592 0.4236
# japanese 0.9631 0.5643
# korean 0.92 0.4549
# marathi 0.3407 0.117
# dutch 0.9886 0.7227
# portuguese 0.9903 0.6566
# tamil 0.7116 0.3451
# telugu 0.6324 0.1909
# turkish 0.7777 0.4412
# urdu 0.8295 0.3566
# vietnamese 0.7956 0.3641
# chinese 0.9502 0.5416

# Start 9
# afrikaans 0.8355 0.2137
# arabic 0.1711 0.2898
# bulgarian 0.4952 0.3311
# greek 0.3426 0.1893
# english 1.0 1.0
# spanish 0.2196 0.2399
# estonian nan nan
# basque 0.2347 0.2653
# persian 0.1959 0.2231
# finnish -0.0294 0.4199
# hebrew nan nan
# hindi 0.1626 0.3285
# hungarian 0.1858 0.1883
# japanese 0.0207 0.1925
# korean 0.098 0.1765
# marathi nan nan
# dutch nan nan
# portuguese -0.0143 0.149
# tamil -0.0496 0.2125
# telugu -0.0298 nan
# turkish nan nan
# urdu 0.0834 0.4385
# vietnamese -0.16 0.3516
# chinese 0.1896 0.2902

# Start 10
# afrikaans 0.8818 0.5704
# arabic 0.7518 0.3991
# bulgarian 0.8111 0.3148
# greek 0.8559 0.5208
# english 1.0 1.0
# spanish 0.767 0.4129
# estonian 0.8408 0.4916
# basque 0.2722 0.3074
# persian 0.912 0.5091
# finnish 0.83 0.5136
# hebrew 0.8701 0.5031
# hindi 0.7868 0.5317
# hungarian 0.5757 0.4601
# japanese 0.7683 0.4143
# korean 0.7471 0.4571
# marathi 0.6477 0.3689
# dutch 0.9331 0.6876
# portuguese 0.8398 0.5045
# tamil 0.8199 0.3245
# telugu 0.6938 0.4236
# turkish 0.6986 0.4321
# urdu 0.8324 0.5428
# vietnamese nan nan
# chinese 0.9255 0.4813

# Start 11
# afrikaans 0.9819 0.7003
# arabic 0.935 0.1404
# bulgarian 0.9708 0.7245
# greek 0.9614 0.5562
# english 1.0 1.0
# spanish 0.9628 0.8966
# estonian 0.9477 0.6167
# basque 0.908 0.3144
# persian 0.9523 0.8764
# finnish 0.9274 0.4188
# hebrew 0.9753 0.7148
# hindi 0.9499 0.8103
# hungarian 0.8781 0.4975
# japanese 0.9554 0.6651
# korean 0.902 0.3333
# marathi 0.8012 0.1522
# dutch 0.9712 0.791
# portuguese 0.9682 0.9052
# tamil 0.8906 0.6354
# telugu 0.8737 0.366
# turkish 0.9255 0.402
# urdu 0.9478 0.7187
# vietnamese 0.0802 0.2404
# chinese 0.9587 0.7117

# Start 12
# afrikaans 0.9815 0.4426
# arabic 0.9736 0.5642
# bulgarian 0.9868 0.548
# greek 0.9743 0.5205
# english 1.0 1.0
# spanish 0.9786 0.6624
# estonian 0.98 0.5778
# basque 0.9719 0.5377
# persian 0.9703 0.5626
# finnish 0.9785 0.5513
# hebrew 0.9843 0.6368
# hindi 0.961 0.5636
# hungarian 0.9572 0.3962
# japanese 0.9602 0.4006
# korean 0.9676 0.5593
# marathi 0.9806 0.3241
# dutch 0.9843 0.6285
# portuguese 0.9815 0.5959
# tamil 0.9643 0.4179
# telugu 0.9782 0.4313
# turkish 0.9813 0.6577
# urdu 0.9527 0.4913
# vietnamese 0.9809 0.3987
# chinese 0.9456 0.3619

# Start 13
# afrikaans 0.913 0.6795
# arabic 0.8008 0.3302
# bulgarian 0.8866 0.4625
# greek 0.8995 0.3306
# english 1.0 1.0
# spanish 0.8317 0.5961
# estonian 0.8151 0.592
# basque nan nan
# persian 0.7559 0.4652
# finnish 0.8928 0.6456
# hebrew 0.8999 0.5097
# hindi 0.7771 0.4704
# hungarian 0.7913 0.5272
# japanese 0.0967 0.2075
# korean 0.3255 0.3743
# marathi 0.2073 0.1974
# dutch 0.9215 0.6527
# portuguese 0.7712 0.5076
# tamil nan nan
# telugu 0.1844 0.1681
# turkish nan nan
# urdu 0.7445 0.4615
# vietnamese -0.0085 0.2074
# chinese nan nan

# Start 14
# afrikaans 0.1284 0.3007
# arabic 0.8721 0.2499
# bulgarian nan nan
# greek -0.0213 0.0294
# english 1.0 1.0
# spanish 0.712 0.2804
# estonian 0.9102 0.3742
# basque 0.2118 0.0301
# persian nan nan
# finnish 0.8828 0.274
# hebrew nan nan
# hindi nan nan
# hungarian nan nan
# japanese 0.6666 0.3915
# korean 0.7768 0.2372
# marathi nan nan
# dutch 0.6691 0.3619
# portuguese 0.8971 0.2122
# tamil nan nan
# telugu nan nan
# turkish nan nan
# urdu nan nan
# vietnamese nan nan
# chinese 0.8467 0.4019

# Start 15
# afrikaans 0.9427 0.6008
# arabic 0.9471 0.6313
# bulgarian 0.9396 0.6472
# greek 0.9361 0.4365
# english 1.0 1.0
# spanish 0.931 0.6699
# estonian 0.8926 0.5975
# basque 0.7501 0.4562
# persian 0.8942 0.4734
# finnish 0.8899 0.5878
# hebrew 0.9563 0.7154
# hindi 0.8885 0.5384
# hungarian 0.878 0.4717
# japanese 0.7743 0.401
# korean 0.6935 0.4326
# marathi 0.7226 0.3817
# dutch 0.9475 0.7482
# portuguese 0.9444 0.7354
# tamil 0.7955 0.3768
# telugu 0.5957 0.4096
# turkish 0.6506 0.46
# urdu 0.8665 0.4563
# vietnamese 0.8358 0.6518
# chinese 0.838 0.5779

# Start 16
# afrikaans 0.4636 0.4698
# arabic 0.4562 0.4667
# bulgarian nan nan
# greek 0.4235 0.3357
# english 1.0 1.0
# spanish 0.7301 0.7159
# estonian 0.4948 0.4836
# basque 0.3427 0.3226
# persian 0.5066 0.3756
# finnish 0.4892 0.5109
# hebrew nan nan
# hindi 0.2246 0.0364
# hungarian 0.6738 nan
# japanese nan nan
# korean 0.4287 0.4623
# marathi nan nan
# dutch 0.7523 0.6815
# portuguese 0.4734 0.4714
# tamil nan nan
# telugu nan nan
# turkish 0.0659 0.0446
# urdu 0.3429 0.1459
# vietnamese 0.0798 0.2049
# chinese 0.507 0.5007

In [ ]:
# # panx-ner

# Start 0
# afrikaans 0.9803 0.8008
# bulgarian 0.9841 0.8509
# bengali 0.9205 0.7176
# german 0.9862 0.8685
# english 1.0 1.0
# spanish 0.9781 0.7756
# estonian 0.9455 0.6421
# basque 0.8994 0.5395
# persian 0.47 0.6359
# finnish 0.8654 0.5607
# french 0.9905 0.9138
# hindi 0.9697 0.7934
# hungarian 0.9328 0.5704
# indonesian 0.8098 0.5337
# italian 0.989 0.879
# javanese 0.969 0.6919
# kazakh 0.8623 0.4465
# marathi 0.8909 0.5169
# dutch 0.9907 0.89
# portuguese 0.9932 0.899
# russian 0.9575 0.804
# swahili 0.9884 0.8151
# telugu 0.9109 0.6005
# tagalog 0.9794 0.825
# turkish 0.9406 0.569
# vietnamese 0.9792 0.7825
# yoruba 0.8285 0.5034

# Start 1
# afrikaans 0.9616 0.7136
# bulgarian 0.9598 0.8688
# bengali 0.9438 0.8512
# german 0.9733 0.8521
# english 1.0 1.0
# spanish 0.9356 0.6195
# estonian 0.9702 0.8341
# basque 0.9515 0.7107
# persian 0.6768 0.5252
# finnish 0.9613 0.7059
# french 0.988 0.8046
# hindi 0.9375 0.836
# hungarian 0.9754 0.7418
# indonesian 0.9497 0.6509
# italian 0.9914 0.8057
# javanese 0.9524 0.725
# kazakh 0.8677 0.6888
# marathi 0.9222 0.796
# dutch 0.9868 0.8268
# portuguese 0.9905 0.8045
# russian 0.9685 0.8957
# swahili 0.7953 0.5098
# telugu 0.803 0.6555
# tagalog 0.9437 0.8704
# turkish 0.965 0.6626
# vietnamese 0.9897 0.938
# yoruba 0.5856 0.7617

# Start 2
# afrikaans 0.9873 0.5732
# bulgarian 0.9791 0.8289
# bengali 0.9281 0.7895
# german 0.9963 0.7859
# english 1.0 1.0
# spanish 0.9759 0.5435
# estonian 0.9959 0.7827
# basque 0.9849 0.5907
# persian 0.5498 0.4551
# finnish 0.9966 0.7234
# french 0.9987 0.8486
# hindi 0.9352 0.6559
# hungarian 0.9941 0.7125
# indonesian 0.9798 0.5868
# italian 0.997 0.7645
# javanese 0.9409 0.5907
# kazakh 0.8551 0.48
# marathi 0.9082 0.6898
# dutch 0.9986 0.8796
# portuguese 0.9953 0.6966
# russian 0.7815 0.6393
# swahili 0.986 0.8569
# telugu 0.7395 0.4875
# tagalog 0.9753 0.8005
# turkish 0.9933 0.6509
# vietnamese 0.9885 0.8883
# yoruba 0.9009 0.6576

# Start 3
# afrikaans 0.9709 0.7568
# bulgarian 0.9689 0.8045
# bengali 0.919 0.7607
# german 0.9857 0.8941
# english 1.0 1.0
# spanish 0.9779 0.9226
# estonian 0.941 0.6315
# basque 0.9425 0.542
# persian 0.8835 0.7261
# finnish 0.8822 0.6799
# french 0.9391 0.8811
# hindi 0.9646 0.8099
# hungarian 0.9515 0.6608
# indonesian 0.9233 0.6356
# italian 0.9836 0.9238
# javanese 0.9327 0.4112
# kazakh 0.9087 0.429
# marathi 0.7185 0.4917
# dutch 0.9875 0.8861
# portuguese 0.9898 0.9201
# russian 0.9581 0.8841
# swahili 0.9809 0.7982
# telugu 0.8919 0.515
# tagalog 0.98 0.8712
# turkish 0.9338 0.7041
# vietnamese 0.9679 0.7387
# yoruba 0.7201 0.5391

# Start 4
# afrikaans 0.9736 0.8316
# bulgarian 0.9517 0.8862
# bengali 0.931 0.8365
# german 0.99 0.9259
# english 1.0 1.0
# spanish 0.9827 0.94
# estonian 0.9606 0.8592
# basque 0.9639 0.8359
# persian 0.9374 0.7735
# finnish 0.9727 0.7126
# french 0.9855 0.9589
# hindi 0.912 0.8759
# hungarian 0.9817 0.8057
# indonesian 0.9937 0.9634
# italian 0.995 0.9684
# javanese 0.9719 0.8243
# kazakh 0.932 0.6927
# marathi 0.89 0.7873
# dutch 0.9937 0.9473
# portuguese 0.9937 0.9607
# russian 0.9727 0.9089
# swahili 0.9534 0.8065
# telugu 0.8718 0.7927
# tagalog 0.9408 0.8916
# turkish 0.9865 0.8037
# vietnamese 0.971 0.883
# yoruba 0.4627 0.7746

# Start 5
# afrikaans 0.9934 0.8098
# bulgarian 0.9769 0.7069
# bengali 0.9275 0.7354
# german 0.9984 0.9414
# english 1.0 1.0
# spanish 0.987 0.9042
# estonian 0.9943 0.7335
# basque 0.9832 0.6164
# persian 0.9292 0.8198
# finnish 0.992 0.7088
# french 0.997 0.9366
# hindi 0.9286 0.6451
# hungarian 0.9941 0.7421
# indonesian 0.9954 0.8921
# italian 0.9953 0.9177
# javanese 0.9865 0.6489
# kazakh 0.9608 0.5632
# marathi 0.895 0.5376
# dutch 0.9957 0.7496
# portuguese 0.9962 0.9272
# russian 0.8583 0.5634
# swahili 0.9851 0.8546
# telugu 0.907 0.6633
# tagalog 0.977 0.8133
# turkish 0.9922 0.6974
# vietnamese 0.9823 0.8191
# yoruba 0.904 0.6109

# Start 6
# afrikaans 0.9849 0.8764
# bulgarian 0.9897 0.9019
# bengali 0.9786 0.8365
# german 0.9854 0.8885
# english 1.0 1.0
# spanish 0.9914 0.7346
# estonian 0.9785 0.8855
# basque 0.9695 0.7399
# persian 0.9192 0.6341
# finnish 0.9869 0.8951
# french 0.9963 0.8909
# hindi 0.9812 0.8013
# hungarian 0.9836 0.8562
# indonesian 0.9835 0.6603
# italian 0.9932 0.9071
# javanese 0.9793 0.7432
# kazakh 0.9617 0.7679
# marathi 0.9711 0.8261
# dutch 0.9907 0.8926
# portuguese 0.9943 0.8793
# russian 0.9866 0.884
# swahili 0.9793 0.7117
# telugu 0.9644 0.7844
# tagalog 0.9927 0.8748
# turkish 0.9844 0.8114
# vietnamese 0.9943 0.8832
# yoruba 0.9854 0.6043

# Relation - Relation

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cov_sim(r1, r2):
    result = np.matmul(r1, r2)
    result /= norm(r1) * norm(r2)
    return result

base_infos = load_pickle('./stats/{}/english.pkl'.format(conf.dataset_name))
for i in list(base_infos.keys()):
    for j in list(base_infos.keys()):
        if i>=j:
            continue
        else:
            print ('Start {}-{}'.format(i, j))
            base_infos = load_pickle('./stats/{}/english.pkl'.format(conf.dataset_name))
            base_mean = base_infos[i][0]['mean'] - base_infos[j][0]['mean']
            base_cov = (base_infos[i][0]['cov'].flatten() + base_infos[j][0]['cov'].flatten()) / 2

            for language in trainer.conf.eval_languages:
                infos = load_pickle('./stats/{}/{}.pkl'.format(conf.dataset_name, language))
                mean = infos[i][0]['mean'] - infos[j][0]['mean']
                cov = (infos[i][0]['cov'].flatten() + infos[j][0]['cov'].flatten()) / 2

                cos_sim = dot(base_mean, mean)
                cos_sim /= (norm(base_mean)*norm(mean))

                cos_sim2 = dot(base_cov, cov)
                cos_sim2 /= (norm(base_cov)*norm(cov))
                # cos_sim2 = cov_sim(base_cov, cov)
                print (language, round(cos_sim, 4), round(cos_sim2, 4))

            print ('')